In [1]:
#Importing all the important packages 

#For accessing and fetching info from the subtitle database file. Provides an SQL interface.
import sqlite3
#For creation of dataframe and saving subtitle info as csv for future retrieval.
import pandas as pd

import io
#For working with paths/directories.
import os

import numpy as np
import glob
import re

from sentence_transformers import SentenceTransformer, util

import joblib
from joblib import Memory

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
def chunking(list_):
    chunk=[]
    chunks=[]
    j=200
    for content in f:
            for i in range(len(content)):
                if (i%200==0) and (i!=0):
                    chunk.append(i)
                    last_one = chunk[-1]
                    
    for content in list_:
        for i in range(len(content)):
            if (i%200==0) and (i>=400):
                chunks.append(content[j-20:i+20])
                j=i
                if j == last_one:
                    chunks.append(content[j-20:])
            elif (i==0):
                chunks.append(content[0:200+20])
    return chunks

In [9]:
path_1 = "data-20240401T135418Z-001/subtitle cleaned/*.srt"

names_=[]
all_docs=[]
for filename in glob.glob(path_1, recursive=True):
    #print(filename)
    #next 3 steps are useful for the last part of this for loop in naming output files.
    #removing the path name 
    name = filename.replace('data-20240401T135418Z-001\\subtitle cleaned/','')
    names_.append(name)
    with open(filename, 'r') as file:
        f = file.read().replace('\n','')
        f =  f.split('\n')
        chunks=chunking(f)
        all_docs.append(chunks)


In [10]:
cachedir = '.cache'
memory = Memory(location=cachedir, verbose=0)

import numpy as np
import torch

#all_batches = []
data_final = all_docs[0:30000]
# Moved outside the loop

for i in range(0, len(data_final), 1000):
    batch = list(data_final[i:i + 1000])
    batch_name = list(names_[i:i + 1000])
    target_directory = "data-20240401T135418Z-001/embeddings/batch_" + str(i) + "to" + str(i + 1000) + "/"
    os.makedirs(target_directory, exist_ok=True)
    for doc, name in zip(batch, batch_name):
        sub_name = name.replace('data-20240401T135418Z-001/subtitle_cleaned\\', '')
        embedding = model.encode(doc)  # Assuming 'model' is defined and 'doc' is your document
        df = pd.DataFrame(embedding)
        df.to_csv(os.path.join(target_directory, sub_name + '.txt'), index=False)

In [5]:
path_2='data-20240401T135418Z-001/embeddings'
embedded_filenames=[]
for root, dirs, files in os.walk(path_2):
    for file in files:
        file=file.replace('.txt','')
        embedded_filenames.append(file)

In [4]:
len(embedded_filenames)

30000

In [6]:
df = pd.DataFrame({'content_name':embedded_filenames})
df_1 = pd.read_csv('data-20240401T135418Z-001/subtitle_connection.csv')

In [7]:
df_1=df_1.drop_duplicates(subset=['content_name'], keep='first')

In [8]:
df_2 = pd.merge(df,df_1, how='inner', on=['content_name'])

In [9]:
df_2.shape

(29990, 3)

In [10]:
df_2.isnull().sum()

content_name    0
ids             0
name            0
dtype: int64

In [9]:
df_2.to_csv('retrieval_file.csv', index=False)

In [10]:
ids= df_2['ids'].values
print(len(ids))

29990


In [2]:
import chromadb 
client=chromadb.PersistentClient(path='chroma')

In [3]:
collection = client.create_collection("final_embed_data", metadata={"hnsw:space": "cosine"})

In [ ]:
import numpy as np

path_2='data-20240401T135418Z-001/embeddings'
for root, dirs, files in os.walk(path_2):
    for file, id_ in zip(files, ids) :
        f = open(root+'/'+file)
        f=f.read()
        list_=list(f.split('\n'))
        list_=list_[1:-1]
        
        for idx,x in enumerate(list_):
            array=[float(y) for y in x.split(',')]
            collection.add(
                embeddings= array,
                ids=str(id_),
                metadatas={str(id_): idx})
        

Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of existing embedding ID: 9317701
Insert of existing embedding ID: 9317701
Add of exis

### Testing with query 

In [26]:
sentence='ice is really good and i love it'

In [27]:
sentence_embeddings= model.encode(sentence)
df_3=pd.DataFrame(sentence_embeddings)

In [28]:
df_3.to_csv(os.path.join('data-20240401T135418Z-001/embed_query/query.txt'), index=False)

In [29]:
query_embedding = [float(y) for y in df_3[0]]

In [30]:
query_embedding[0:10]

[-0.14534848928451538,
 -0.059601154178380966,
 0.021183500066399574,
 0.029214506968855858,
 0.007402131799608469,
 0.056056492030620575,
 0.05688067525625229,
 0.022361645475029945,
 0.03225700557231903,
 0.014428253285586834]

In [31]:
print(type(query_embedding))
query_ans= collection.query(
    query_embeddings= query_embedding  ,
    n_results=5
)

<class 'list'>


In [32]:
query_ans

{'ids': [['9331728.0', '9184397.0', '9521858.0', '9521862.0', '9446318.0']],
 'distances': [[0.9864174127578735,
   0.9864174127578735,
   0.9864174127578735,
   0.9864174127578735,
   0.9864174127578735]],
 'metadatas': [[{'9331728.0': 0},
   {'9184397.0': 0},
   {'9521858.0': 0},
   {'9521862.0': 0},
   {'9446318.0': 0}]],
 'embeddings': None,
 'documents': [['0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,1